In [176]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [177]:
raw_df = pd.read_csv('Data/Kindle.csv')

# Cleaning Data

In [178]:
#Loại bỏ dữ liệu trùng lặp
raw_df.drop_duplicates(inplace= True)

In [179]:
#In ra thông tin dữ liệu của file Kindle.csv
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   post_id                        564 non-null    int64  
 1   text                           564 non-null    object 
 2   post_text                      564 non-null    object 
 3   shared_text                    19 non-null     object 
 4   original_text                  0 non-null      float64
 5   time                           564 non-null    object 
 6   timestamp                      564 non-null    int64  
 7   image                          491 non-null    object 
 8   image_lowquality               564 non-null    object 
 9   images                         564 non-null    object 
 10  images_description             564 non-null    object 
 11  images_lowquality              564 non-null    object 
 12  images_lowquality_description  564 non-null    obj

In [180]:
#Đếm số lượng giá trị bị thiếu trong mỗi cột của dữ liệu
raw_df.isnull().sum()

post_id                            0
text                               0
post_text                          0
shared_text                      545
original_text                    564
time                               0
timestamp                          0
image                             73
image_lowquality                   0
images                             0
images_description                 0
images_lowquality                  0
images_lowquality_description      0
video                            515
video_duration_seconds           564
video_height                     564
video_id                         515
video_quality                    564
video_size_MB                    564
video_thumbnail                  515
video_watches                    564
video_width                      564
likes                              0
comments                           0
shares                             0
post_url                           0
link                              23
l

Trong dữ liệu có nhiều cột chúng ta không có dự định phân tích hoặc bị thiếu dữ liệu quá nhiều nên không phân tích được, nên cần thực hiện bước tiền xử lý

# Tiền xử lý

In [181]:
clean_df = pd.DataFrame(raw_df)
clean_df.columns

Index(['post_id', 'text', 'post_text', 'shared_text', 'original_text', 'time',
       'timestamp', 'image', 'image_lowquality', 'images',
       'images_description', 'images_lowquality',
       'images_lowquality_description', 'video', 'video_duration_seconds',
       'video_height', 'video_id', 'video_quality', 'video_size_MB',
       'video_thumbnail', 'video_watches', 'video_width', 'likes', 'comments',
       'shares', 'post_url', 'link', 'links', 'user_id', 'username',
       'user_url', 'is_live', 'factcheck', 'shared_post_id', 'shared_time',
       'shared_user_id', 'shared_username', 'shared_post_url', 'available',
       'comments_full', 'reactors', 'w3_fb_url', 'reactions', 'reaction_count',
       'with', 'page_id', 'sharers', 'image_id', 'image_ids', 'was_live',
       'fetched_time', 'start_url'],
      dtype='object')

In [182]:
# Loại bỏ các cột bị trùng lặp hoặc không sử dụng trong quá trình phân tích dữ liệu
clean_df = clean_df.drop(['text','shared_text', 'original_text', 'image', 'image_lowquality',
                         'images', 'images_description', 'images_lowquality',
                         'images_lowquality_description', 'video', 'video_duration_seconds',
                         'video_height', 'video_id', 'video_quality', 'video_size_MB',
                         'video_thumbnail', 'video_watches', 'video_width', 'user_id', 
                         'link', 'links', 'username', 'user_url', 'is_live', 'factcheck', 'shared_post_id', 
                         'shared_time', 'shared_user_id', 'shared_username', 'shared_post_url', 
                         'available', 'w3_fb_url', 'with', 'page_id', 'sharers', 'image_id', 
                         'image_ids', 'was_live','start_url'], axis='columns')

In [183]:
#kiểm tra một lần nữa các cột dữ liệu còn lại để chắc chắn không loại bỏ nhầm dữ liệu.
clean_df.columns

Index(['post_id', 'post_text', 'time', 'timestamp', 'likes', 'comments',
       'shares', 'post_url', 'comments_full', 'reactors', 'reactions',
       'reaction_count', 'fetched_time'],
      dtype='object')

In [184]:
#kiểm tra xem có bao nhiêu hàng trong cột "comments_full" chứa kí tự trống "[]" và xoá đi những hàng đó.
count_empty_rows = (clean_df['comments_full'] == "[]").sum()
clean_df = clean_df[clean_df['comments_full'] != "[]"]

In [185]:
#kiểm tra xem có bao nhiêu post trùng nhau và xoá đi những post bị trùng đó.
count_duplicate_rows = clean_df.duplicated('post_id').sum()
clean_df = clean_df[~clean_df.duplicated('post_id')]

chuyển đổi múi giờ sang giờ usa

In [186]:

# Convert the time column from string to datetime
clean_df['time'] = pd.to_datetime(clean_df['time'])

# Define the time difference between GMT+7 and USA time (Eastern Standard Time, GMT-5)
time_difference = timedelta(hours=-12)

# Convert the time to USA time by adding the time difference
clean_df['time'] = clean_df['time'] + time_difference


In [187]:
#kiểm tra xem sau quá trình tiền xử lý, các cột dữ liệu có cột nào bị khuyết thiếu dữ liệu không? 
#nếu có phải thực hiện loại bỏ các hàng có dữ liệu bị thiếu đó.
clean_df.isnull().sum()

post_id           0
post_text         0
time              0
timestamp         0
likes             0
comments          0
shares            0
post_url          0
comments_full     0
reactors          0
reactions         0
reaction_count    0
fetched_time      0
dtype: int64

In [188]:
#xác định hình dạng của tập dữ liệu xem gồm bao nhiêu hàng, bao nhiêu cột
clean_df.shape

(348, 13)

Sau quá trình tiền xử lý, dữ liệu đã cơ bản sạch sẽ 

# Làm sạch dữ liệu

In [189]:
#Chuyển đổi "chuỗi thời gian" thành kiểu "datetime".
clean_df['time'] = pd.to_datetime(clean_df['time'])
clean_df['day'] = clean_df['time'].dt.day

In [190]:
#Chuyển đổi mỗi phần tử trong reactions thành một từ điển.
clean_df['reactions'] = clean_df['reactions'].apply(lambda x : dict(eval(x))) 
#Chuyển đổi từng loại reaction thành các cột Dataframe.
list_reactions = clean_df['reactions'].apply(pd.Series) 
#thay thế mọi giá trị bị thiếu (NaN) trong "list_reactions" DataFrame bằng số 0.
list_reactions = list_reactions.fillna(0)
#kết hợp DataFrame gốc "clean_df" với "list_reactions" DataFrame và xóa cột reactions ban đầu.
clean_df = pd.concat([clean_df, list_reactions], axis=1).drop('reactions', axis=1) 

# xuất file dữ liệu đã xử lý


In [191]:
path = 'Data/Kindle_Cleaned.csv'
clean_df.to_csv(path, index=False)